In [1]:
import torch

from torchvision.transforms import ToTensor, Compose, Normalize

from data.dataset import get_data

from model.model import YOLONet
from model.train import train
from model.loss import DetectionLoss

DATASET = "../datasets/Human-Fall-Detection"
BATCH_SIZE = 8

SUBSET = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
transform_data = Compose([ToTensor(), Normalize((0.4379,), (0.3040,))])

data = get_data(DATASET, transform_data, device=device, batch_size=BATCH_SIZE, end=BATCH_SIZE)

train_loader, val_loader, test_loader = list(data["dataloaders"].values())

In [9]:
with torch.device(device):
  model = YOLONet(ch=(64,32,32))
  # model.load_state_dict(torch.load('model_e4.pt'))
  criterion = DetectionLoss(model.head)

train(model, train_loader, val_loader, test_loader, 1, criterion)

  0%|          | 0/337 [00:00<?, ?it/s]c:\Users\yapzh\miniconda3\envs\dlenv\Lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  3%|▎         | 10/337 [02:30<1:36:37, 17.73s/it]

[1, 10] train loss: tensor([3.2190, 5.5430, 4.1840], device='cuda:0')


  6%|▌         | 20/337 [05:01<1:33:44, 17.74s/it]

[1, 20] train loss: tensor([3.3231, 4.7452, 4.0227], device='cuda:0')


  9%|▉         | 30/337 [07:31<1:30:32, 17.69s/it]

...

[4, 20] train loss: tensor([2.5546, 2.9926, 2.8035], device='cuda:0')


  9%|▉         | 30/337 [10:23<2:05:14, 24.48s/it]

[4, 30] train loss: tensor([2.5370, 3.0507, 2.8093], device='cuda:0')


 11%|█         | 36/337 [12:49<1:47:13, 21.37s/it]


KeyboardInterrupt: 

In [10]:
torch.save(model.state_dict(), 'model.pt')